In [1]:
#Import library
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import feature_column
import pandas as pd
import joblib
from tensorflow.keras.models import save_model
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder


In [2]:
# Dataset load
df = pd.read_csv('Dataset/dataset_cleaned1.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18595 entries, 0 to 18594
Data columns (total 14 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Maker                                18595 non-null  object 
 1   Model                                18595 non-null  object 
 2   Number_of_Cylinders                  18595 non-null  int64  
 3   Engine_Type                          18595 non-null  int64  
 4   Engine_Horse_Power                   18595 non-null  float64
 5   Engine_Horse_Power_RPM               18595 non-null  int64  
 6   Transmission                         18595 non-null  int64  
 7   Mixed_Fuel_Consumption_per_100_km_l  18595 non-null  float64
 8   Fuel_Tank_Capacity                   18595 non-null  int64  
 9   Acceleration_0_to_100_Km             18595 non-null  float64
 10  Max_Speed_Km_per_Hour                18595 non-null  int64  
 11  Fuel_Grade                  

In [4]:
#Required features for training ? (tentative)
req_feat = ['Number_of_Cylinders',
            'Engine_Type',
            'Engine_Horse_Power',
            'Engine_Horse_Power_RPM',
            'Transmission',
            'Acceleration_0_to_100_Km',
            'Fuel_Grade',
            'Mixed_Fuel_Consumption_per_100_km_l']

target = 'Mixed_Fuel_Consumption_per_100_km_l'

#Data for train
data = df[req_feat]

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18595 entries, 0 to 18594
Data columns (total 8 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   Number_of_Cylinders                  18595 non-null  int64  
 1   Engine_Type                          18595 non-null  int64  
 2   Engine_Horse_Power                   18595 non-null  float64
 3   Engine_Horse_Power_RPM               18595 non-null  int64  
 4   Transmission                         18595 non-null  int64  
 5   Acceleration_0_to_100_Km             18595 non-null  float64
 6   Fuel_Grade                           18595 non-null  int64  
 7   Mixed_Fuel_Consumption_per_100_km_l  18595 non-null  float64
dtypes: float64(3), int64(5)
memory usage: 1.1 MB


In [6]:
#Data split into val and train set
train_df, val_df = train_test_split(data, test_size=0.1)
train_label = train_df.pop('Mixed_Fuel_Consumption_per_100_km_l')
val_label = val_df.pop('Mixed_Fuel_Consumption_per_100_km_l')
print(f'Number of training datasets: {len(train_df)}')
print(f'Number of validation datasets: {len(val_df)}')

Number of training datasets: 16735
Number of validation datasets: 1860


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16735 entries, 12602 to 15564
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Number_of_Cylinders       16735 non-null  int64  
 1   Engine_Type               16735 non-null  int64  
 2   Engine_Horse_Power        16735 non-null  float64
 3   Engine_Horse_Power_RPM    16735 non-null  int64  
 4   Transmission              16735 non-null  int64  
 5   Acceleration_0_to_100_Km  16735 non-null  float64
 6   Fuel_Grade                16735 non-null  int64  
dtypes: float64(2), int64(5)
memory usage: 1.0 MB


In [8]:
# Standardize the data
columns_to_standardize = ['Engine_Horse_Power',
                          'Engine_Horse_Power_RPM',
                          'Acceleration_0_to_100_Km']
scaler = StandardScaler()
train_df[columns_to_standardize] = scaler.fit_transform(train_df[columns_to_standardize])
val_df[columns_to_standardize] = scaler.transform(val_df[columns_to_standardize])

In [9]:
# One Hot Encode
categorical_columns = ['Number_of_Cylinders',
            'Engine_Type',
            'Transmission',
            'Fuel_Grade']
transformer = make_column_transformer(
    (OneHotEncoder(), categorical_columns),
    remainder='passthrough')
train_data = transformer.fit_transform(train_df)
train_df = pd.DataFrame(
    train_data, 
    columns=transformer.get_feature_names_out())
val_data = transformer.transform(val_df)
val_df = pd.DataFrame(
    val_data, 
    columns=transformer.get_feature_names_out())


In [12]:
# Convert dataframe to tensorflow datasets
train_data = tf.data.Dataset.from_tensor_slices((train_df.values, train_label.values))
val_data = tf.data.Dataset.from_tensor_slices((val_df.values, val_label.values))

In [13]:
#Batches and shuffles (for train only) the data
train_data = train_data.shuffle(10).batch(32)
val_data = val_data.batch(32)

In [15]:
#Create model and train

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model = tf.keras.Sequential([
  tf.keras.layers.Dense(160, activation='relu', input_shape=(19,)),
  tf.keras.layers.Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='mse',
              metrics=['mae'])

model.fit(train_data,
          validation_data=val_data,
          epochs=50,
          callbacks=[stop_early])


Epoch 1/50


523/523 [==============================] - 2s 2ms/step - loss: 6.8092 - mae: 1.7284 - val_loss: 1.9826 - val_mae: 1.0431
Epoch 2/50
523/523 [==============================] - 1s 2ms/step - loss: 1.6824 - mae: 0.9527 - val_loss: 1.4532 - val_mae: 0.8993
Epoch 3/50
523/523 [==============================] - 1s 2ms/step - loss: 1.4378 - mae: 0.8849 - val_loss: 1.3422 - val_mae: 0.8674
Epoch 4/50
523/523 [==============================] - 1s 2ms/step - loss: 1.3690 - mae: 0.8651 - val_loss: 1.2884 - val_mae: 0.8497
Epoch 5/50
523/523 [==============================] - 1s 2ms/step - loss: 1.3330 - mae: 0.8526 - val_loss: 1.2533 - val_mae: 0.8369
Epoch 6/50
523/523 [==============================] - 1s 2ms/step - loss: 1.3103 - mae: 0.8443 - val_loss: 1.2245 - val_mae: 0.8264
Epoch 7/50
523/523 [==============================] - 1s 2ms/step - loss: 1.2944 - mae: 0.8386 - val_loss: 1.2149 - val_mae: 0.8241
Epoch 8/50
523/523 [==============================] - 1s 2ms/step - loss:

In [17]:
# Save model in a h5 file
save_model(model, "car_regress.h5")

# Save scaler in a joblib file
joblib.dump(scaler, 'scaler.joblib')

# Save transformer in a joblib file
joblib.dump(transformer, 'transformer.joblib')

C:\Users\akane\AppData\Local\Temp\ipykernel_6604\3562494085.py:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(model, "car_regress.h5")


['transformer.joblib']

In [21]:
data_test = [[8, 0, 0, 2, 500, 5600, 6.7]] #Toyota Sequioa
data_test = pd.DataFrame(data_test, columns=['Number_of_Cylinders', 
                                                            'Engine_Type',
                                                            'Transmission',
                                                            'Fuel_Grade',
                                                            'Engine_Horse_Power',
                                                            'Engine_Horse_Power_RPM',
                                                            'Acceleration_0_to_100_Km'])
data_test[columns_to_standardize] = scaler.transform(data_test[columns_to_standardize])
data_test.info()
data_test = transformer.transform(data_test)
data_test= tf.data.Dataset.from_tensor_slices(data_test)
data_test = data_test.batch(1)
hasil = model.predict(data_test)
print(hasil)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Number_of_Cylinders       1 non-null      int64  
 1   Engine_Type               1 non-null      int64  
 2   Transmission              1 non-null      int64  
 3   Fuel_Grade                1 non-null      int64  
 4   Engine_Horse_Power        1 non-null      float64
 5   Engine_Horse_Power_RPM    1 non-null      float64
 6   Acceleration_0_to_100_Km  1 non-null      float64
dtypes: float64(3), int64(4)
memory usage: 184.0 bytes
1/1 [==============================] - 0s 717us/step


[[15.88987]]
